# 32. 나라면 이런 거 사겠다(3) - 행복전략

[32강의](https://www.youtube.com/watch?v=k_khgxL4pLs) 를 파이썬으로 구현해보겠습니다.

## 설치

Pandas 가 있어야 합니다. 설치가 안되어있으시면 다음 셀을 실행시키세요.

In [1]:
import sys
!{sys.executable} -m pip install pandas xlrd openpyxl

## 다운로드

[네이버 카페 퀀트킹(QuantKing)](https://cafe.naver.com/quantking)에서 최신 엑셀 파일을 다운받으세요.

저는 **QuantKing(2019.01.25).zip** 파일을 받았습니다. 받으면 확장자 **xlsb** 파일이 보입니다. 엑셀로 여셔서 **QuantKing.xlsx** 파일로 다시 저장해주세요.

## 파일 로딩

파일을 열어서 조사해 보겠습니다. Pandas의 read_excel() 이란 함수를 이용하면 열 수 있네요.

In [2]:
import pandas as pd

In [3]:
filepath = 'QuantKing.xlsx'
df = pd.read_excel(filepath, sheet_name='퀀트데이타', skiprows=2).drop('Unnamed: 0', axis=1)
df.head()

,코드 번호,회사명,업종 (대),업종 (소),코스피코스닥,주가 (원),시가총액 (억),상장주식수 (만주),자사주 (만주),자사주 비중 (%),...,18년 2Q.2,18년 3Q.2,18년 4Q(E).2,19년 1Q(E).2,18년4Q 매출액,18년4Q 영업이익,18년4Q 순이익,18년4Q 매출액.1,18년4Q 영업이익.1,18년4Q 순이익.1
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,44750,2671478,596978.0,0,0.0,...,109815,129674,100441.00,79168.0,590000.00,108000.00,NaN,631283.0,133354.0,100441.0
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,74600,543090,72800.0,4400,6.0,...,43299,46936,33962.90,16797.0,99380.81,44300.75,33962.90,101549.0,50660.0,40819.0
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,128500,274564,21367.0,939,4.4,...,7006,2692,-1297.74,7829.0,256695.34,5011.27,-1297.74,261108.0,7611.0,6727.0
3,A068270,셀트리온,제약 및 바이오,바이오,코스피,208500,266772,12795.0,85,0.7,...,840,562,544.00,521.0,NaN,NaN,NaN,2512.0,792.0,544.0
4,A207940,삼성바이오로직스,제약 및 바이오,바이오,코스피,403000,266645,6617.0,0,0.0,...,-96,-301,4362.00,28.0,NaN,NaN,NaN,1393.0,149.0,4362.0


강의에서 사용하는 컬럼만 잘라서 보관하도록 하겠습니다.

In [4]:
df = df[['회사명',
         '업종\n(대)',
         '과거\nPCR',
         '배당\n성향\n(%)',
         '시가\n배당률\n(%)'
        ]].set_index('회사명')
df.head()

,업종 (대),과거 PCR,배당 성향 (%),시가 배당률 (%)
회사명,,,,
삼성전자,반도체 관련장비 및 부품,4.03,12.3,1.9
SK하이닉스,반도체 관련장비 및 부품,2.71,6.4,1.3
현대차,자동차 및 관련부품,5.42,20.3,3.1
셀트리온,제약 및 바이오,60.02,0.6,0.0
삼성바이오로직스,제약 및 바이오,472.92,0.0,0.0


## 행복전략이란?

돈 잘 벌고 배당 많이 주는 회사 주식 매수!

- PCR 상위 30% (어느 정도 저평가!)
    - PCR: 시가총액 / 영업현금흐름
    - 영업현금흐름: '회계의 기초 (영상 29)' 참고
    
- 배당성향: 35~75%
    - 배당성향 자체가 우량주 지표! 한국은 배당 매우 적게 줌
    
- 그런 주식 중 배당 수익률 (배당 / 주가) 가장 높은 주식 매수

- 배당수익률: 배당이 줄지만 않으면 그 정도 현금수익 확정!

- 중요: 이 회사는 배당을 계속 유지하는 역사가 있는가?

In [5]:
df = df[df['과거\nPCR'] != 0]
df['1/PCR'] = 1 / df['과거\nPCR']

# 상위 30% 1/PCR
thirty = int(df.shape[0] * 0.3)
df = df.nlargest(thirty, '1/PCR')

# 배당성향 35 ~ 75%
df = df[df['배당\n성향\n(%)'] >= 35]
df = df[df['배당\n성향\n(%)'] <= 75]

# 시가배당률로 정렬
df = df.sort_values(by='시가\n배당률\n(%)', ascending=False)
df.head(n=20)

,업종 (대),과거 PCR,배당 성향 (%),시가 배당률 (%),1/PCR
회사명,,,,,
유아이엘,IT 장비 및 소재,3.08,68.6,11.0,0.324675
삼양옵틱스,IT 장비 및 소재,7.05,73.3,10.6,0.141844
오렌지라이프,금융,1.65,57.8,8.0,0.606061
휴켐스,화학,6.40,53.2,6.9,0.156250
KPX홀딩스,화학,6.42,39.1,6.4,0.155763
엔브이에이치코리아,자동차 및 관련부품,5.36,73.3,6.0,0.186567
한전산업,유틸리티,3.66,46.1,5.3,0.273224
지역난방공사,유틸리티,1.41,46.7,4.9,0.709220
에스텍,전기 및 전자기기,8.45,42.5,4.8,0.118343


5분 이후에 회사 하나하나 분석하시면서 설명해주십니다. 꼭 더 시청하세요.

<span style="color:red">**중요: 내가 하는 방법이 진리라는거 아닙니다!**</span> 

- 언제 매수매도: 주식 수익률은 11~5월에 월등 높습니다.
    - 한국 뿐 아니라 전 세계!
    - KOSPI 수익의 120%가 11~5월에 발생!!
    
- 강환국 투자: 게으름의 극치: 그냥 11월 매수, 4월말 매도!

- 익절: 절대 안함

- 손절: 한달에 한번 정도 들어가서 -20% 이하 주식은 매도

- 트레일링 스탑: 최고점에서 20% 미끄러진 놈 매도
    - 이론적으로 좋으나 게을러서 안함
    
- 물타기: 내 사전에 손절은 있어도 물타기는 없다

- 마켓타이밍: 절대모멘텀 적용
    - 모멘텀이 너무 좋아 2018년에는 4월에 안팔고 6월에 팜

- 평균모멘텀스코어: 최근 1, 2, 3, ... 12개월 모멘텀 체크 (지수)
    - 1월: 1, 2, 3개월 모멘텀이 +로 전환, 자산의 1/4 투입!
    - 3주에 걸쳐서 "나라면 이런거 사겠다" 주식들 매수!
    - 그냥 내년부터는 11~5월 "남자 전략"으로 컴백?